In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import csv
import pandas as pd
import re

In [2]:
# time
t1 = time.time()
time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t1))


'2019-05-19 21:30:37'

In [3]:
bidInfo = {}

In [4]:
csvFile = open("namelist.csv","w+")
writer = csv.writer(csvFile)

bidInfo = {}

header = ("name","date","title","consultPrice","currentPrice","bidCount","delayCount","applyCount","area","areaB","url")

writer.writerow(header)

#writer.writerow(('name','url'))



89

In [5]:
# tianjin
# sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2" \
#                  "&st_param=-1&auction_start_from=2018-01-01&auction_start_to=2019-05-13&spm=a213w.3064813.9001.2"  


# wuhan
sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&city=%CE%E4%BA%BA&sorder=2" \
                  "&st_param=-1&auction_start_from=2018-01-01&auction_start_to=2019-05-19&spm=a213w.3064813.9001.2"

In [6]:
# Start the driver
driver = webdriver.Chrome()
# Setup wait for later
wait = WebDriverWait(driver, 10)

driver.get(sourceUrl)

# Store the ID of the original window
original_window = driver.current_window_handle

# Check we don't have other windows open already
assert len(driver.window_handles) == 1

In [7]:
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))


<selenium.webdriver.remote.webelement.WebElement (session="86de68793739bc8ecb3133e72e446082", element="0.11989565798922652-1")>

In [8]:
#if test, set  pages = 1 
#pages = 1
pages = driver.find_element_by_class_name("page-total").text

In [9]:
for page in range(1,int(pages)+1):
    if(page > 1):
        driver.find_element_by_link_text(str(page)).click()
        time.sleep(1)
    
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sf-pai-item-list")))
    data = driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML')
    jsonData =  json.loads(data)
    items = jsonData["data"]
    
    itemLen = len(items)
    #itemLen = 6   #test
    for item in range(itemLen):    
        bidInfo["status"] = items[item]["status"]

        
        if(bidInfo["status"] == "done"):
            
            bidInfo["id"] = items[item]["id"]
            bidInfo["title"] = items[item]["title"]
            bidInfo["consultPrice"] = items[item]["consultPrice"]
            bidInfo["currentPrice"] = items[item]["currentPrice"]
            bidInfo["bidCount"] = items[item]["bidCount"]
            bidInfo["delayCount"] = items[item]["delayCount"]
            bidInfo["applyCount"] = items[item]["applyCount"]

            url = "#pai-item-"+str(bidInfo["id"])
            # Click the link which opens in a new window
            driver.find_element_by_css_selector(url).click()
            time.sleep(1) # have to wait sometime

            # Wait for the new window or tab
            wait.until(EC.number_of_windows_to_be(2))
            handles = driver.window_handles
            driver.switch_to.window(handles[1])
            # size
            
            urlNew = driver.current_url
            url2 = str(urlNew)+"#J_Confirmation"
            driver.get(url2)
            
            
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrap")))
            info = driver.find_element_by_class_name("c-content").text
            index = info.find("通过")
            
            # getName
            bidInfo["name"] = info[4:index]
            # getDate
            bidInfo["date"] = driver.find_element_by_class_name("c-stamp").text
            

            #print
            print("Name: "+bidInfo["name"]+": Page :"+str(page)+"   Item :"+str(item))



           # get area
            driver.get(str(urlNew)+"#NoticeDetail")
            time.sleep(1)
            notice = driver.find_element_by_id("J_NoticeDetail").get_attribute('textContent')            
            left = notice.find("建筑")
           # right = notice.find("方米",left)
            
            if(left != -1 ):
                size = notice[(left+2) : (left+22)]
                areas = re.findall(r"\d+\.\d*",size)
                if(len(areas) > 0):
                    bidInfo["area"]  = areas[0]
                else:
                    bidInfo["area"]  = 0
            else:
                bidInfo["area"]  = 0

            

           ###  Tip
            driver.get(str(urlNew)+"#RemindTip")
            time.sleep(1)
            tip  = driver.find_element_by_id("J_desc").get_attribute('textContent')
            tip_left = tip.find("建筑")
           # tip_right =  tip.find("方米",tip_left)
            
            if(tip_left != -1):
                size = tip[(tip_left+2) : (tip_left+22)]           
                areaB = re.findall(r"\d+\.\d*",size)
                if(len(areaB) > 0 ):
                    bidInfo["areaB"] =areaB[0]
                else:
                    bidInfo["areaB"] = 0
            else:
                bidInfo["areaB"] = 0

                
                
            bidInfo["url"] = urlNew   # add url

### write into file
            writer.writerow((bidInfo["name"],bidInfo["date"],bidInfo["title"],
                                   bidInfo["consultPrice"],bidInfo["currentPrice"],
                                   bidInfo["bidCount"],bidInfo["delayCount"],bidInfo["applyCount"],
                                   bidInfo["area"],bidInfo["areaB"],bidInfo["url"]
                                 ))
                                 

            driver.close()
            driver.switch_to.window(handles[0])
        else:
            pass
           # print("***item failed******"+": Page :"+str(page)+"   Item :"+str(item))



Name: 阎韬: Page :1   Item :0
Name: 黄立: Page :1   Item :3
Name: 蹇本兵: Page :1   Item :10
Name: 胡运爽: Page :1   Item :14
Name: 杨帆: Page :1   Item :15
Name: 祝加新: Page :1   Item :22
Name: 陈爱梅: Page :1   Item :24
Name: 库嫱: Page :1   Item :28
Name: 帅猛章: Page :1   Item :30
Name: 喻三元: Page :1   Item :31
Name: 杨金芝: Page :2   Item :0
Name: 陈章溶: Page :2   Item :1
Name: 陈舒: Page :2   Item :2
Name: 张文海: Page :2   Item :7
Name: 周元学: Page :2   Item :8
Name: 黄小慧: Page :2   Item :9
Name: 陈娜: Page :2   Item :18
Name: 蔡桂松: Page :2   Item :21
Name: 王慧: Page :2   Item :22
Name: 陈金和: Page :3   Item :13
Name: 李建民: Page :3   Item :14
Name: 郑平: Page :5   Item :18
Name: 武汉台北商场有限公司: Page :5   Item :19
Name: 王伽司: Page :5   Item :26
Name: 杨杰: Page :5   Item :29
Name: 李亚红: Page :5   Item :31
Name: 周元学: Page :6   Item :0
Name: 李琼华: Page :6   Item :1
Name: 湖北宝中龙观旅游集团有限公司: Page :6   Item :3
Name: 孙琴花: Page :6   Item :6
Name: 赵仕琼: Page :6   Item :7
Name: 刘锦玉: Page :6   Item :8
Name: 王友爱: Page :6   Item :11
Name: 胡洪超: Page

Name: 黄韬: Page :36   Item :21
Name: 张瑞兰: Page :36   Item :22
Name: 张志勇: Page :36   Item :23
Name: 宋文: Page :36   Item :25
Name: 梅英: Page :36   Item :30
Name: 陈治国: Page :36   Item :31
Name: 田芳: Page :37   Item :4
Name: 黄鸿: Page :37   Item :5
Name: 陈磊: Page :37   Item :6
Name: 赵三红: Page :37   Item :28
Name: 陈飘: Page :37   Item :30
Name: 赵艾: Page :38   Item :2
Name: 宋宝萍: Page :38   Item :8
Name: 朱振资: Page :38   Item :9
Name: 李岚: Page :38   Item :10
Name: 李青: Page :38   Item :18
Name: 柴炎军: Page :38   Item :22
Name: 张雪梅: Page :38   Item :25
Name: 包雪梅: Page :38   Item :26
Name: 余晶晶: Page :38   Item :29
Name: 孙进: Page :38   Item :30
Name: 刘浪: Page :39   Item :5
Name: 杨军: Page :39   Item :6
Name: 崔贞桢: Page :39   Item :8
Name: 毛钧毅: Page :39   Item :13
Name: 孙改玲: Page :39   Item :14
Name: 柴小汇: Page :39   Item :19
Name: 杨瑞环: Page :39   Item :20
Name: 姜治明: Page :39   Item :31
Name: 程丽君: Page :40   Item :5
Name: 黄运闪: Page :40   Item :7
Name: 王凡: Page :40   Item :10
Name: 李龙: Page :40   Item :16
Nam

Name: 邹志明: Page :58   Item :8
Name: 蔡伟: Page :58   Item :11
Name: 何淑珍: Page :58   Item :12
Name: 黄松平: Page :58   Item :16
Name: 邹志明: Page :58   Item :18
Name: 代波: Page :58   Item :20
Name: 曹川川: Page :58   Item :22
Name: 陈智慧: Page :58   Item :25
Name: 张明: Page :58   Item :27
Name: 董永怀: Page :58   Item :28
Name: 陈耀行: Page :59   Item :2
Name: 汪向松: Page :59   Item :3
Name: 夏东霞: Page :59   Item :4
Name: 程雁: Page :59   Item :5
Name: 江玲: Page :59   Item :7
Name: 陈敦义: Page :59   Item :8
Name: 杨帆: Page :59   Item :10
Name: 刘杰: Page :59   Item :12
Name: 袁丹志: Page :59   Item :13
Name: 武汉宇泽文商业咨询有限公司: Page :59   Item :14
Name: 陈小英: Page :59   Item :20
Name: 王中来: Page :59   Item :21
Name: 赵耀: Page :59   Item :30
Name: 陈丽红: Page :59   Item :31
Name: 肖云祖: Page :60   Item :0
Name: 冯腾: Page :60   Item :1
Name: 胡娟: Page :60   Item :3
Name: 毛加荣: Page :60   Item :4
Name: 李雪: Page :60   Item :5
Name: 张甲: Page :60   Item :10
Name: 付海涛: Page :60   Item :14
Name: 闵志武: Page :60   Item :15
Name: 郁麟: Page :60   I

In [10]:
driver.quit()

In [11]:
csvFile.close()
csv = pd.read_csv('namelist.csv', encoding='utf-8')
csv.to_excel('namelist.xlsx', sheet_name='data')

In [12]:
t2 = time.time()
time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t2))

seconds = t2 - t1
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print ("%02d:%02d:%02d" % (h, m, s))


00:52:49


In [13]:
#            print("URL: "+driver.current_url)
#             print("id:"+str(bidInfo["id"])+" title: "+bidInfo["title"])
#             print("consultPrice:"+str(bidInfo["consultPrice"]) + " currentPrice:" +str(bidInfo["currentPrice"]))
#             print("bidCount:"+str(bidInfo["bidCount"]) +" delayCount:"+str(bidInfo["delayCount"])+" applyCount:"+str(bidInfo["applyCount"]))
#             print("date:"+bidInfo["date"])

In [14]:
#^[1-9]\d*\.\d*|0\.\d*[1-9]\d*$


In [15]:
# match = r"\d+\.\d*"

# areas = re.findall(match,"abc12121223ac-1.12dddd")

# areas

